Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    Наименование вакансии.
    Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
    Ссылку на саму вакансию.
    Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [104]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [144]:
def parsing_hh(vacancy):

    vacancies = []
    url = 'https://hh.ru/search/vacancy'

    params = {
            'text': vacancy,
            'search_field': 'name',
            'items_on_page': '100',
            'page': '1',
        }

    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0'
        }

    html = requests.get(url, params=params, headers=headers)
    
    if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
            
            for item in vacancy_items:
                        vacancies.append(parsing_item_hh(item))
    
    return vacancies

In [158]:
def parsing_item_hh(item):

    vacancies = {}
    
    # vacancy_name
    vacancy_name = item.find('div', {'class': 'vacancy-serp-item__info'}) \
                        .getText()
    
    vacancies['vacancy_name'] = vacancy_name
    
    #salary
    salary = item.find('div', {'class': 'vacancy-serp-item__sidebar'})
    
    if not salary:
        salary_min = None
        salary_max = None

    else:
        salary = salary.getText() \
                        .replace(u'\u202f', u'')
        salary = re.split(r'\s|-', salary)
        
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[2])            
        

    vacancies['salary_min'] = salary_min
    vacancies['salary_max'] = salary_max

    # link
       
    vacancy_link = item.find('div', {'class': 'vacancy-serp-item__info'}) \
                        .find('a')['href']
    
    vacancies['vacancy_link'] = vacancy_link 
    
    # site
    vacancies['site'] = 'hh.ru'
    
    return vacancies

In [159]:
def parser_vacancy(vacancy):
        
    vacancies = []
    vacancies.extend(parsing_hh(vacancy))
    
    df = pd.DataFrame(vacancies)

    return df

In [161]:
vacancy = 'плотник'
df = parser_vacancy(vacancy)



df.head()

,vacancy_name,salary_min,salary_max,vacancy_link,site
0,Плотник-бетонщик,60000.0,70000.0,https://lobnya.hh.ru/vacancy/46259577?from=vac...,hh.ru
1,Плотник,28000.0,28000.0,https://lobnya.hh.ru/vacancy/46288554?from=vac...,hh.ru
2,Упаковщик (плотник),51500.0,70000.0,https://lobnya.hh.ru/vacancy/41486781?from=vac...,hh.ru
3,Плотник,20000.0,NaN,https://lobnya.hh.ru/vacancy/49470393?from=vac...,hh.ru
4,Плотник 4-5 разряда,25000.0,NaN,https://lobnya.hh.ru/vacancy/44007998?from=vac...,hh.ru


In [162]:
df.to_csv('vacancies_hh.csv', sep=";", index=False)